# GameDay2020 Notebook

The standard dilemma of fantasy baseball is who to choose next: the best available player but in a position you've already filled?; the best at a rare position?; the next best closer because they're going fast?  Whatever choice you make will have consequences, as the next player will no longer have your pick as a choice, and so on.  Ideally you would forecast how each of those would play out and take the best option --- that's exactly what this code does: it identifies the best player left in each of the positions you still need to fill, simulates the rest of draft in each of those scenarios, and returns the best choice.  

And it's easy to use!  After [cloning](https://github.com/wrapgenius/GameDay2020) GameDay2020 (and provided you have Jupyter Notebooks with Python 3 installed, and a spreadsheet program) you should have everything you need to do a live roto draft.

The code is based on three ingredients: *projections*, *rankings*, and *injuries*:  
- Projections are estimates of player performances for 2020, and are based on data scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips) (model = ZiPS, Steamer, or TheBat.) 
- Rankings are draft-order recommendations.  The default (i.e.; ranking_method = 'Yahoo') is to use Yahoo Rankings (from Feb 22, 2020).  Others include [ESPN](https://www.espn.com/fantasy/baseball/story/_/id/28285423/fantasy-baseball-2020-rankings-roto-rotisserie), [FantasyPros](https://www.fantasypros.com/mlb/rankings/overall.php), and [Rotographs](https://fantasy.fangraphs.com), or you can put in your own.  Yahoo for yahoo leagues makes most sense since it most closely imitates autodraft.    
- Injured players to be removed from draft consideration are located in the Injuries2020.xlsx file found in the Injured_List_Spreadsheets directory. 

_Your_ roto league may have more or less than 12 teams, or use different stats; that ok!  Declare them when defining the Draft object: most of the obvious stats are included, more obscure ones may require hacking into the [fangraphs_projection_2020](https://github.com/wrapgenius/GameDay2020/blob/master/GameDayFunctions/fangraphs_projection_2020.py).  Note, it can't _yet_ do AL or NL only.

If you choose to shuffle_picks = True, it will not default to the next best pick, but draw a random number between 0 and 1 and compare it to the sigmoid function 1/1+exp(-(pick_number - avg_pick_number)).  So say it's the 15th pick and Trevor Story, who is typically drafted 10th is available, there is an 85% chance you'll select him.  

Note, the *search_depth* is how many levels of player to compare.  For example, if the only roster positions left to fill are 1B and SS, a search depth of 2 will compare simulations of the top 2 1B and the top 2 SS.  Increasing *search_depth* slows it down a little, but makes better predictions.  

And it really does work; you'll see from the simulations below just how much it blows the competition away.    

In [1]:
# Load packages.  Must have pandas and numpy.  

import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import copy
from GameDayFunctions.fangraphs_projection_2020 import Projection
from GameDayFunctions.draft_2020 import Draft
from GameDayFunctions.gameday_simulation_2020 import Simulation

First define the year, the paths, the search depth, and whether the simulation should simply pick the next player in the list, or shuffle to add some randomness.   

In [2]:
year = 2021
path_data = "projections/"
path_drafts = "Draft_Pick_Spreadsheets/"
shuffle_picks = False
search_depth = 3

First need a **Projection** object.  Projections are scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips), and include ZiPS, Steamer, and TheBat.  
In this example I create a dictionary of the three projections.  

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS',ranking_method = 'FantasyPros')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer',ranking_method = 'FantasyPros')
#player_ranking_thebat = Projection(path_data=path_data,year=year,model='TheBat',ranking_method = 'FantasyPros')
#player_rankings = {"ZiPS":player_ranking_zips, "Steamer":player_ranking_steam, "TheBat":player_ranking_thebat}
#player_rankings = {"Steamer":player_ranking_steam, "TheBat":player_ranking_thebat}
player_rankings = {"Steamer":player_ranking_steam, "ZiPS":player_ranking_zips}
#player_rankings = {"ZiPS":player_ranking_zips}

projections/2021/ZiPS_2021_Hitters.csv
projections/2021/ZiPS_2021_Pitchers.csv
projections/2021/Steamer_2021_Hitters.csv
projections/2021/Steamer_2021_Pitchers.csv


Next need a **Draft** object.  The default, e.g.; Draft(player_rankings['ZiPS']), uses 12 teams and the roster spots/roto stats shown in this example. 

In [4]:
# Enter your roto-league-specific stats.  The default settings are shown here.  
draft_position = 2
number_teams = 12
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}
batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] 

# Choose the projection system you prefer ()
player_projections = player_rankings['ZiPS']

# Get an instance of the Draft Class with your league-specific details and projection preference.
zips_example = Draft(player_projections, 
                     draft_position = draft_position, 
                     number_teams = number_teams,
                     roster_spots = roster_spots,
                     batter_stats = batter_stats,
                     pitcher_stats = pitcher_stats)

## There are two ways to use this code, **Live-Drafting** and **Simulating**.  

We start with some **live-drafting** examples, which is how you'd use the code to draft a winning team ;)

To use while drafting in real-time, while it's happening you must record the draft results in an *.xlsx formatted spreadsheet* (so if you're using Numbers, need to export to Excel).  Once it's saved, simply confirm the filename is recorded in draft_pick_file and run the cell.  

Examples below show results for after 2, 3, and 4 rounds.  Notice that at each stage the projected placement in the standings improves.  

In [ ]:
zSim, zRank = Simulation(number_sims = 2)

projections/2020/ZiPS_2020_Pitchers.csv
projections/2020/ZiPS_2020_Hitters.csv
Sim 0
Sim 1
> /home/marcoviero/Repositories/GameDay2020/GameDayFunctions/gameday_simulation_2020.py(137)write_simulation_results()
    135         file_pickle = self.path_sims + self.projection_type + '_' + self.ranking_method + '_' + str(self.number_teams) + '_teams_' + str(self.number_sims) + '_sims'
    136         pdb.set_trace()
--> 137         outfile = open(file_pickle,'wb')
    138         pickle.dump(simulation_results,outfile)
    139         outfile.close()



ipdb>  file_pickle


'simulations/ZiPS_FantasyPros_12_teams_2_sims'


ipdb>  pwd


*** NameError: name 'pwd' is not defined


...

{1: {0: (8, 'Gerrit Cole', 'SP', 1, 149.5), 1: (5, 'Mookie Betts', 'OF', 2, 144.5), 2: (26, 'Stephen Strasburg', 'SP', 1, 147.5), 3: (43, 'Clayton Kershaw', 'SP', 1, 167.0), 4: (12, 'Juan Soto', 'OF', 1, 167.0), 5: (73, 'Eugenio Suarez', '3B', 1, 185.0), 6: (27, 'Yordan Alvarez', 'OF', 1, 188.0), 7: (111, 'Jorge Polanco', 'SS', 1, 192.0), 8: (31, 'Charlie Blackmon', 'OF', 1, 192.0), 9: (288, 'Yadier Molina', 'C', 1, 192.5), 10: (91, 'Trevor Bauer', 'SP', 1, 192.5), 11: (78, 'Victor Robles', 'OF', 1, 184.5), 12: (197, 'Christian Walker', '1B', 1, 187.0), 13: (131, 'Madison Bumgarner', 'SP', 1, 186.5), 14: (180, 'Gavin Lux', '2B', 1, 186.5), 15: (274, 'Seth Lugo', 'RP', 1, 188.0), 16: (277, 'Scott Oberg', 'RP', 1, 190.5)}, 2: {0: (17, 'Jose Ramirez', '3B', 1, 158.5), 1: (40, 'Ketel Marte', '2B', 1, 165.5), 2: (12, 'Juan Soto', 'OF', 5, 138.5), 3: (154, 'Mitch Garver', 'C', 3, 145.5), 4: (103, 'Carlos Carrasco', 'RP', 3, 142.5), 5: (95, 'Jeff McNeil', '2B', 6, 137.5), 6: (101, 'Mike Moust

ipdb>  self.number_sims


2


ipdb>  simulation_results[2]


*** KeyError: 2


In [7]:
# Example 5th pick in the 2nd round
search_depth = 1
draft_pick_file = 'TestPicksround2.xlsx'
zipsT = Draft(player_rankings['Steamer'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(search_depth = search_depth, draft_pick_file = draft_pick_file, path_list = path_drafts)

Finding Best Pick For Team 5
Stored Result for Pick 0 [20/7] Trea Turner  SS whose placement/score is 4/141.5
Stored Result for Pick 1 [20/8] Gerrit Cole  SP whose placement/score is 6/138.5
Stored Result for Pick 2 [20/11] Jose Ramirez  3B whose placement/score is 7/130.0
Stored Result for Pick 3 [20/15] Bryce Harper  OF whose placement/score is 9/110.0
Stored Result for Pick 4 [20/28] Jose Abreu  1B whose placement/score is 4/138.5
Stored Result for Pick 5 [20/30] Ozzie Albies  2B whose placement/score is 3/141.5
Stored Result for Pick 6 [20/51] J.T. Realmuto  C whose placement/score is 10/116.5
Stored Result for Pick 7 [20/54] Kenta Maeda  RP whose placement/score is 12/97.5
Best Pick is Ozzie Albies  putting you in 3 place


##### DJ LeMahieu?  Really?  Yes, trust the process.  

The Projected_roto_stats contains the estimated results from all teams given the recommented pick

In [6]:
projected_stats = projected_roto_stats[0]
projected_standings = projected_roto_stats[3]
projected_place = projected_roto_stats[4]
projected_scores = projected_roto_stats[5]

In [7]:
# The projected_stats give the raw numbers

projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,801,743,246,30,238,847,114,643,0.258288,0.819398,63,45,5,5,50,268,997,3.881099,1.194433,4
1,822,809,290,23,276,907,100,550,0.272758,0.845637,66,47,3,3,94,267,957,3.974228,1.244686,9
2,773,749,261,23,257,858,47,574,0.260279,0.820616,74,51,5,5,88,309,1183,3.850824,1.198089,8
3,642,667,220,23,208,649,102,383,0.264646,0.806873,59,42,5,5,39,321,996,3.807559,1.237211,3
4,774,793,284,39,260,851,95,539,0.265066,0.822090,57,34,3,3,147,231,872,3.615506,1.171777,13
5,799,794,298,30,247,876,78,544,0.271874,0.838610,60,46,5,5,74,234,922,3.917170,1.183148,7
6,725,733,251,24,224,745,116,516,0.258639,0.800304,64,46,4,4,79,300,1039,3.715524,1.186661,7
7,642,615,212,24,205,729,51,441,0.270840,0.843441,61,37,4,4,72,302,1036,3.623602,1.193646,6
8,728,799,241,30,234,801,151,500,0.259679,0.787378,64,44,5,5,101,286,936,3.788859,1.223433,8
9,815,845,264,30,237,826,127,534,0.270727,0.820628,68,44,4,4,68,277,952,3.767319,1.202197,6


In [9]:
# And projected_scores show how they translate to roto scores

projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,11.0,5.0,8.0,9.5,12.0,12.0,4.0,12.0,2.0,12.0,3.0,5.0,9.0,9.0,6.0,12.0,7.0,7.0,8.0,6.5
1,10.0,8.0,7.0,3.0,9.0,8.0,12.0,10.0,4.0,3.0,9.5,1.0,12.0,12.0,4.0,1.0,12.0,10.0,7.0,10.0
2,8.0,12.0,11.0,11.0,5.0,9.0,8.0,7.0,10.0,4.0,11.0,2.0,9.0,9.0,11.0,11.0,6.0,5.0,9.0,2.0
3,5.0,6.0,5.0,8.0,3.0,3.0,9.0,4.0,8.0,2.0,4.5,8.5,5.0,5.0,10.0,4.0,3.0,3.0,3.0,4.5
4,7.0,11.0,6.0,6.5,4.0,5.0,2.0,6.0,12.0,8.0,4.5,11.5,5.0,5.0,9.0,10.0,4.0,8.0,6.0,3.0
5,3.0,3.0,4.0,1.0,8.0,6.0,1.0,5.0,6.0,9.0,8.0,3.5,9.0,9.0,1.5,3.0,8.0,6.0,5.0,10.0
6,6.0,9.0,9.0,4.5,6.0,7.0,11.0,8.0,9.0,5.0,6.5,11.5,9.0,9.0,7.0,2.0,9.0,11.0,10.0,6.5
7,12.0,10.0,10.0,12.0,10.0,11.0,5.5,11.0,11.0,10.0,1.0,8.5,1.0,1.0,5.0,7.0,1.0,1.0,1.0,10.0
8,4.0,1.0,3.0,6.5,7.0,4.0,10.0,3.0,5.0,11.0,6.5,3.5,2.5,2.5,3.0,9.0,5.0,4.0,4.0,10.0
9,1.0,4.0,1.0,2.0,1.0,2.0,3.0,1.0,7.0,1.0,12.0,6.5,5.0,5.0,12.0,5.5,10.0,12.0,11.0,1.0


In [8]:
# Finally, the projected_standings shows the resulting standings

projected_standings

5     157.0
4     151.5
9     148.0
1     147.0
0     139.0
2     133.5
10    132.5
8     123.0
11    116.5
6     112.0
7     109.5
3      90.5
dtype: float64

In [9]:
# But if you just want to know how your team finishes, that's projected_place

projected_place

2

## Next we simulate complete drafts. 

*Naive* drafting is a quick way to build rosters by simply picking the next eligible (i.e., open roster spot) player on the ranked list. 

The default it to not use naive_draft, rather, each time draft_position comes up, it cycles through the top *search_depth* level of players at each open roster position.  For example, if *search_depth* = 2 and it is the first round, it will compare 2 players in every position.  

In [12]:
# Lets begin with a super-quick draft: 4 teams, limited roster spots, and naive selections (i.e., next available pick on ranked list) picks.

naive_draft = True
zipsS= Draft(player_rankings['ZiPS'],draft_position = 2,number_teams = 4, roster_spots = {'C':0,'1B':0,'2B':1, '3B':1,'SS':1,'OF':2,'UTIL':1,'SP':1,'RP':1,'P':1,'BN':1})
zipsS.draft_all(naive_draft = naive_draft, shuffle_picks = shuffle_picks)
print('Naive Placement = '+str(zipsS.roto_placement))

Naive Placement = 4


In [14]:
# Now try a naive draft again, but for full 12 team and all 24 roster spots.  Draft Position is still 2 

naive_draft = True
zipsNaive_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zipsNaive_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Naive Placement  = '+str(zipsNaive_2.roto_placement))

Naive Placement  = 9


In [15]:
# Placement is not great.  But we can look at the roster, which looks ok, can we really do that much better?  

zipsNaive_2.teams[2-1]['roster']

{'OF': [[array(['Mookie Betts '], dtype=object),
   array(['Aaron Judge '], dtype=object)],
  array(['Trent Grisham '], dtype=object)],
 'SP': [array(['Walker Buehler '], dtype=object),
  array(['Aaron Nola '], dtype=object)],
 'P': [[array(['Blake Snell '], dtype=object),
   array(['Zack Wheeler '], dtype=object)],
  array(['Patrick Corbin '], dtype=object)],
 'UTIL': array(['Nick Castellanos '], dtype=object),
 '1B': array(['Max Muncy '], dtype=object),
 'RP': [array(['Kenley Jansen '], dtype=object),
  array(['Rafael Montero '], dtype=object)],
 'SS': array(['Didi Gregorius '], dtype=object),
 '3B': array(['Josh Donaldson '], dtype=object),
 '2B': array(['Nick Madrigal '], dtype=object),
 'C': array(['Austin Nola '], dtype=object),
 'BN': array(['Max Kepler '], dtype=object)}

In [16]:
# We can look at the standings, and see team 1 (draft_position-1) is right in the middle of the pack.  

zipsNaive_2.roto_standings

9     153.0
10    153.0
2     145.0
8     140.0
4     138.5
5     136.5
6     130.0
11    122.0
1     120.0
0     115.0
3     113.0
7      94.0
dtype: float64

In [29]:
steam_4.teams[4-1]['roster']

{'OF': [[array(['Mookie Betts '], dtype=object),
   array(['George Springer '], dtype=object)],
  array(['J.D. Martinez '], dtype=object)],
 'RP': [array(['Aroldis Chapman '], dtype=object),
  array(['Kevin Gausman '], dtype=object)],
 'SP': [array(['Shane Bieber '], dtype=object),
  array(['Zack Wheeler '], dtype=object)],
 '3B': array(['Rafael Devers '], dtype=object),
 'C': array(['Sean Murphy '], dtype=object),
 '2B': array(['Ian Happ '], dtype=object),
 'UTIL': array(['Charlie Blackmon '], dtype=object),
 'SS': array(['Carlos Correa '], dtype=object),
 'P': [[array(['Charlie Morton '], dtype=object),
   array(['Giovanny Gallegos '], dtype=object)],
  array(['Greg Holland '], dtype=object)],
 'BN': array(['Tommy Pham '], dtype=object),
 '1B': array(['Eric Hosmer '], dtype=object)}

In [27]:
dir(steam_4)
steam_4.roto_team_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,805,765,247,21,267,802,118,615,0.257570,0.815737,60,57,5,5,49,347,1068,4.200473,1.279280,4
1,674,668,214,15,215,665,106,466,0.258300,0.803565,71,62,6,6,103,374,1169,4.155675,1.276222,10
2,750,751,246,14,238,751,62,578,0.264477,0.822940,55,52,5,5,50,314,996,4.178011,1.258924,4
3,888,890,304,28,275,862,94,579,0.267297,0.820934,58,53,5,5,181,295,959,4.089314,1.260433,16
4,838,831,277,21,262,812,84,622,0.260695,0.810402,51,46,4,4,77,309,893,3.973564,1.281383,6
5,860,826,288,26,283,862,75,669,0.263300,0.830525,61,58,5,5,60,301,1002,4.367183,1.272208,5
6,778,797,256,25,241,763,93,538,0.259117,0.794872,72,56,5,5,94,313,1124,4.045179,1.227654,8
7,764,791,255,22,242,781,113,542,0.256197,0.790631,52,41,4,4,197,305,957,3.647657,1.225340,16
8,763,760,248,19,259,785,77,537,0.261488,0.817794,64,54,5,5,145,357,1055,4.149880,1.288516,13
9,847,827,262,22,253,773,113,672,0.258403,0.807553,51,43,3,3,71,273,782,4.227254,1.298841,6


In [28]:
# This time do the full-simulation, i.e., with naive_draft = False
# Remember, it will try every remaining open position and pick the one whose resulting placement is highest.  
# Setting silent = False shows the decision logic for each pick (highest placement followed by highest score [placement, score])

naive_draft = False
steam_4 = Draft(player_rankings['Steamer'], draft_position = 4)
steam_4.draft_all(naive_draft = naive_draft, search_depth = 2, shuffle_picks = True, silent=False)
print('Placement  = '+str(steam_4.roto_placement))

Team 1 Drafting Ronald Acuna
Team 2 Drafting Jacob deGrom 
Team 3 Drafting Freddie Freeman 
Stored Result for Pick 0 [4/2] Mookie Betts  OF whose placement/score is 6/136.0
Stored Result for Pick 1 [4/3] Fernando Tatis SS whose placement/score is 8/119.0
Stored Result for Pick 2 [4/4] Mike Trout  OF whose placement/score is 9/118.5
Stored Result for Pick 3 [4/7] Trea Turner  SS whose placement/score is 11/107.0
Stored Result for Pick 4 [4/8] Gerrit Cole  SP whose placement/score is 9/116.5
Stored Result for Pick 5 [4/11] Jose Ramirez  3B whose placement/score is 10/103.0
Stored Result for Pick 6 [4/12] Shane Bieber  SP whose placement/score is 8/122.5
Stored Result for Pick 7 [4/14] Cody Bellinger  1B whose placement/score is 12/107.5
Stored Result for Pick 8 [4/17] Manny Machado  3B whose placement/score is 9/114.5
Stored Result for Pick 9 [4/28] Jose Abreu  1B whose placement/score is 12/99.0
Stored Result for Pick 10 [4/30] Ozzie Albies  2B whose placement/score is 12/112.0
Stored R

In [5]:
# This time do the full-simulation, i.e., with naive_draft = False
# Remember, it will try every remaining open position and pick the one whose resulting placement is highest.  
# Setting silent = False shows the decision logic for each pick (highest placement followed by highest score [placement, score])

naive_draft = False
zips_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zips_2.draft_all(naive_draft = naive_draft, search_depth = 2, shuffle_picks = shuffle_picks,silent=False)
print('Placement  = '+str(zips_2.roto_placement))

Team 1 Drafting Christian Yelich
Stored Result for Pick 0 [2/2] Mookie Betts OF whose placement/score is 6/138.5
Stored Result for Pick 1 [2/3] Mike Trout OF whose placement/score is 6/137.0
Stored Result for Pick 2 [2/6] Anthony Rendon 3B whose placement/score is 4/132.0
Stored Result for Pick 3 [2/7] Cody Bellinger 1B whose placement/score is 4/142.0
Stored Result for Pick 4 [2/8] Freddie Freeman 1B whose placement/score is 5/137.0
Stored Result for Pick 5 [2/9] Alex Bregman 3B whose placement/score is 4/149.0
Stored Result for Pick 6 [2/11] Justin Verlander SP whose placement/score is 4/137.0
Stored Result for Pick 7 [2/13] Jacob deGrom SP whose placement/score is 4/137.0
Stored Result for Pick 8 [2/14] Francisco Lindor SS whose placement/score is 5/138.0
Stored Result for Pick 9 [2/15] Whit Merrifield 2B whose placement/score is 5/134.0
Stored Result for Pick 10 [2/17] Trevor Story SS whose placement/score is 3/142.0
Stored Result for Pick 11 [2/19] Jose Altuve 2B whose placement/s

In [24]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_roster = zips_2.teams[2-1]['roster']# draft_position -1 because zero based
projected_stats = zips_2.roto_team_stats
projected_standings = zips_2.roto_standings
projected_place = zips_2.roto_placement
projected_scores = zips_2.roto_team_stats_rank

In [25]:
projected_place

1

In [30]:
# Look at the roster and it is not markedly different from Naive, but the results beg to differ.  

projected_roster

{'SS': array(['Trevor Story'], dtype=object),
 'SP': [array(['Gerrit Cole'], dtype=object),
  array(['Shane Bieber'], dtype=object)],
 '3B': array(['Rafael Devers'], dtype=object),
 'OF': [[array(['Aaron Judge'], dtype=object),
   array(['Adam Eaton'], dtype=object)],
  array(['Marcell Ozuna'], dtype=object)],
 'UTIL': array(['Manny Machado'], dtype=object),
 'BN': array(['Tommy Edman'], dtype=object),
 '2B': array(['Gleyber Torres'], dtype=object),
 'P': [[array(['Chris Paddack'], dtype=object),
   array(['Sergio Romo'], dtype=object)],
  array(['Dustin May'], dtype=object)],
 'C': array(['J.T. Realmuto'], dtype=object),
 'RP': [array(['Chris Martin'], dtype=object),
  array(['Craig Stammen'], dtype=object)],
 '1B': array(['Christian Walker'], dtype=object)}

In [16]:
projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,720.0,790.0,262.0,28.0,219.0,745.0,98.0,492.0,0.272076,0.821402,55.0,37.0,4.0,4.0,27.0,241.0,1008.0,3.588769,1.146309,2.0
1,929.0,995.0,320.0,42.0,265.0,913.0,116.0,546.0,0.276151,0.822852,46.0,29.0,3.0,3.0,50.0,188.0,780.0,3.498711,1.124196,5.0
2,769.0,848.0,265.0,33.0,193.0,669.0,149.0,490.0,0.275204,0.807540,47.0,35.0,2.0,2.0,17.0,241.0,794.0,3.772834,1.205395,1.0
3,627.0,620.0,224.0,23.0,196.0,671.0,50.0,460.0,0.265500,0.825660,67.0,41.0,4.0,4.0,31.0,312.0,1143.0,3.495938,1.154816,3.0
4,506.0,573.0,178.0,13.0,140.0,501.0,47.0,379.0,0.279400,0.828933,41.0,29.0,2.0,2.0,67.0,194.0,670.0,3.833686,1.209472,5.0
5,628.0,675.0,222.0,22.0,198.0,667.0,94.0,464.0,0.271670,0.829042,36.0,22.0,1.0,1.0,42.0,216.0,707.0,3.509932,1.187927,3.0
6,721.0,797.0,262.0,24.0,227.0,749.0,74.0,526.0,0.274110,0.830180,35.0,24.0,1.0,1.0,50.0,186.0,604.0,3.501016,1.206735,4.0
7,710.0,752.0,252.0,31.0,214.0,734.0,87.0,451.0,0.274532,0.828095,56.0,33.0,3.0,3.0,23.0,243.0,968.0,3.434494,1.150922,2.0
8,740.0,764.0,261.0,28.0,215.0,740.0,88.0,514.0,0.273117,0.829282,60.0,39.0,2.0,2.0,67.0,277.0,1003.0,3.571324,1.180071,5.0
9,782.0,744.0,268.0,23.0,260.0,827.0,94.0,611.0,0.271100,0.856808,45.0,32.0,1.0,1.0,53.0,218.0,723.0,3.615973,1.220705,4.0


In [17]:
projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,7.0,9.0,8.5,8.0,9.0,9.0,10.0,8.0,5.0,3.0,8.0,4.0,11.5,11.5,4.0,4.5,11.0,5.0,10.0,10.0
1,12.0,12.0,12.0,12.0,12.0,12.0,11.0,11.0,11.0,4.0,5.0,9.5,9.0,9.0,7.5,11.0,5.0,9.0,12.0,3.0
2,10.0,11.0,10.0,11.0,3.0,5.0,12.0,7.0,9.0,2.0,6.0,5.0,6.0,6.0,1.0,4.5,7.0,2.0,4.0,12.0
3,3.0,3.0,4.0,3.5,4.0,6.0,2.5,5.0,1.0,5.0,12.0,1.0,11.5,11.5,5.0,1.0,12.0,10.0,8.0,7.5
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,12.0,8.0,3.0,9.5,6.0,6.0,10.5,10.0,2.0,1.0,2.0,3.0
5,4.0,4.0,3.0,2.0,5.0,4.0,8.5,6.0,4.0,9.0,2.0,12.0,2.5,2.5,6.0,9.0,3.0,7.0,5.0,7.5
6,8.0,10.0,8.5,5.0,10.0,10.0,4.0,10.0,7.0,11.0,1.0,11.0,2.5,2.5,7.5,12.0,1.0,8.0,3.0,5.5
7,6.0,7.0,6.0,10.0,7.0,7.0,6.0,4.0,8.0,7.0,9.0,7.0,9.0,9.0,2.0,3.0,9.0,11.0,9.0,10.0
8,9.0,8.0,7.0,8.0,8.0,8.0,7.0,9.0,6.0,10.0,10.0,2.5,6.0,6.0,10.5,2.0,10.0,6.0,6.0,3.0
9,11.0,6.0,11.0,3.5,11.0,11.0,8.5,12.0,3.0,12.0,4.0,8.0,2.5,2.5,9.0,8.0,4.0,4.0,1.0,5.5


In [18]:
projected_standings

1     189.0
0     156.0
7     146.0
8     142.0
9     137.5
6     137.5
2     133.5
3     116.5
10    108.5
5     106.0
11    105.5
4      82.0
dtype: float64

#### At 189 points, this team demolishes the competition!

In [17]:
# Now lets compare Naive and Full Simulation, starting with the 7th pick, for ZiPS and Steamer

naive_draft = True
zipsNaive_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zipsNaive_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
steamerNaive_7 = Draft(player_rankings['Steamer'], draft_position = 7)
steamerNaive_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('ZiPS Naive Placement  = '+str(zipsNaive_7.roto_placement))
print('Steamer Naive Placement  = '+str(steamerNaive_7.roto_placement))

ZiPS Naive Placement  = 8
Steamer Naive Placement  = 5


In [19]:
# Full sim with ZiPS

naive_draft = False
zips_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zips_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Zips Full Simulation Placement  = '+str(zips_7.roto_placement))

Zips Full Simulation Placement  = 1


In [20]:
# Full sim with Steamer

naive_draft = False
steamer_7 = Draft(player_rankings['Steamer'], draft_position = 7)
steamer_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Steamer Full Simulation Placement  = '+str(steamer_7.roto_placement))

Steamer Full Simulation Placement  = 1


In [32]:
# And for the 11th pick.  Naive picks first...

naive_draft = True
zipsNaive_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zipsNaive_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('ZiPS Naive Placement  = '+str(zipsNaive_11.roto_placement))

ZiPS Naive Placement   = 7


In [33]:
# Now full sim with ZiPS

naive_draft = False
zips_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zips_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Zips Full Simulation Placement  = '+str(zips_11.roto_placement))

Zips Full Simulation Placement  = 1


In [35]:
zips_11_projected_roster = zips_11.teams[11-1]['roster']# draft_position -1 because zero based
zips_11_projected_stats = zips_11.roto_team_stats
zips_11_projected_standings = zips_11.roto_standings
zips_11_projected_place = zips_11.roto_placement
zips_11_projected_scores = zips_11.roto_team_stats_rank

In [36]:
zips_11_projected_standings

10    190.0
0     155.0
7     152.5
6     144.5
1     143.0
8     140.5
2     132.0
9     119.5
11    115.0
3     104.5
4      90.5
5      73.0
dtype: float64

In [21]:
# And Full sim with Steamer

naive_draft = False
steamer_11 = Draft(player_rankings['Steamer'], draft_position = 11)
steamer_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Steamer Full Simulation Placement  = '+str(steamer_11.roto_placement))

Steamer Full Simulation Placement  = 1


In [37]:
steamer_11_projected_roster = steamer_11.teams[11-1]['roster']# draft_position -1 because zero based
steamer_11_projected_stats = steamer_11.roto_team_stats
steamer_11_projected_standings = steamer_11.roto_standings
steamer_11_projected_place = steamer_11.roto_placement
steamer_11_projected_scores = steamer_11.roto_team_stats_rank

In [38]:
steamer_11_projected_standings

10    175.0
7     142.5
4     142.0
3     140.5
5     140.0
11    137.5
0     134.0
9     123.0
2     115.5
1     110.0
6     105.5
8      94.5
dtype: float64

#### It's clear that the sim wins every time, and it's not even close.  